# Finjustering av Open AI-modeller

Den här notebooken bygger på den aktuella vägledningen i [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst)-dokumentationen från Open AI.

Finjustering förbättrar prestandan hos grundmodeller för din applikation genom att träna om modellen med extra data och kontext som är relevant för just det användningsområdet eller scenariot. Observera att tekniker som _few shot learning_ och _retrieval augmented generation_ gör det möjligt att förstärka den ursprungliga prompten med relevant data för att höja kvaliteten. Dessa metoder begränsas dock av den maximala token-fönsterstorleken för den aktuella grundmodellen.

Med finjustering tränar vi i praktiken om själva modellen med den data som behövs (vilket gör att vi kan använda många fler exempel än vad som får plats i token-fönstret) – och vi kan sedan använda en _anpassad_ version av modellen som inte längre behöver få exempel vid inferenstid. Det gör inte bara att vår promptdesign blir mer effektiv (vi får större frihet att använda token-fönstret till annat), utan kan också sänka våra kostnader (eftersom vi skickar färre tokens till modellen vid inferenstid).

Finjustering sker i fyra steg:
1. Förbered och ladda upp träningsdata.
1. Kör träningsjobbet för att få en finjusterad modell.
1. Utvärdera den finjusterade modellen och iterera för att förbättra kvaliteten.
1. Distribuera den finjusterade modellen för inferens när du är nöjd.

Observera att inte alla grundmodeller kan finjusteras – [se OpenAI:s dokumentation](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) för den senaste informationen. Du kan även finjustera en modell som redan har finjusterats tidigare. I den här guiden använder vi `gpt-35-turbo` som vår grundmodell för finjustering.

---


### Steg 1.1: Förbered ditt dataset

Vi ska bygga en chatbot som hjälper dig att förstå det periodiska systemet genom att svara på frågor om ett grundämne med en limerick. I _den här_ enkla guiden skapar vi bara ett dataset för att träna modellen med några exempel på svar som visar det förväntade formatet på datan. I ett verkligt scenario skulle du behöva skapa ett dataset med många fler exempel. Det kan också vara möjligt att använda ett öppet dataset (för ditt användningsområde) om det finns, och formatera om det för att använda vid finjustering.

Eftersom vi fokuserar på `gpt-35-turbo` och vill ha ett enskilt svar (chat completion) kan vi skapa exempel med [detta föreslagna format](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) som följer OpenAIs krav för chat completion. Om du förväntar dig konversationer med flera turer, ska du använda [formatet för flera turer](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) som innehåller en `weight`-parameter för att ange vilka meddelanden som ska användas (eller inte) vid finjusteringen.

Vi använder det enklare formatet för enskilda turer i den här guiden. Datan är i [jsonl-format](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) med 1 post per rad, där varje post är ett JSON-objekt. Koden nedan visar 2 poster som exempel – se [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) för hela exempelsamlingen (10 exempel) som vi använder i vår finjusteringsguide. **Obs:** Varje post _måste_ skrivas på en enda rad (inte uppdelad på flera rader som i en formaterad JSON-fil)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

I ett verkligt scenario behöver du ett mycket större antal exempel för att få bra resultat – det blir en avvägning mellan svarskvalitet och tid/kostnad för finjusteringen. Vi använder ett litet dataset för att snabbt kunna slutföra finjusteringen och visa processen. Se [detta exempel i OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) för en mer avancerad guide till finjustering.


### Steg 1.2 Ladda upp din datamängd

Ladda upp datan med hjälp av Files API [enligt beskrivningen här](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Observera att för att kunna köra den här koden måste du först ha gjort följande:
 - Installerat Python-paketet `openai` (se till att du använder version >=0.28.0 för de senaste funktionerna)
 - Satt miljövariabeln `OPENAI_API_KEY` till din OpenAI API-nyckel
För att läsa mer, se [Installationsguiden](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) som tillhandahålls för kursen.

Kör nu koden för att skapa en fil att ladda upp från din lokala JSONL-fil.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Steg 2.1: Skapa finjusteringsjobbet med SDK:n


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Steg 2.2: Kontrollera statusen för jobbet

Här är några saker du kan göra med `client.fine_tuning.jobs`-API:  
- `client.fine_tuning.jobs.list(limit=<n>)` - Lista de senaste n finjusteringsjobben  
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Hämta detaljer om ett specifikt finjusteringsjobb  
- `client.fine_tuning.jobs.cancel(<job_id>)` - Avbryt ett finjusteringsjobb  
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Lista upp till n händelser från jobbet  
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Det första steget i processen är _att validera träningsfilen_ för att säkerställa att datan har rätt format.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Steg 2.3: Spåra händelser för att följa framsteg


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Steg 2.4: Visa status i OpenAI-instrumentpanelen


Du kan också se statusen genom att besöka OpenAI:s webbplats och gå till avsnittet _Fine-tuning_ på plattformen. Där visas statusen för det aktuella jobbet, och du kan även följa historiken för tidigare körningar. I den här skärmdumpen ser du att den tidigare körningen misslyckades, medan den andra lyckades. För att ge lite bakgrund, så inträffade detta när den första körningen använde en JSON-fil med felaktigt formaterade poster – när detta rättades till, slutfördes den andra körningen utan problem och modellen blev tillgänglig för användning.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.sv.png)


Du kan också se statusmeddelanden och mätvärden genom att scrolla ner längre i den visuella instrumentpanelen som visas här:

| Meddelanden | Mätvärden |
|:---|:---|
| ![Meddelanden](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.sv.png) |  ![Mätvärden](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.sv.png)|


### Steg 3.1: Hämta ID & Testa finjusterad modell i kod


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Steg 3.2: Ladda & testa finjusterad modell i Playground

Du kan nu testa den finjusterade modellen på två sätt. För det första kan du besöka Playground och använda rullgardinsmenyn för modeller för att välja din nyligen finjusterade modell från listan. Det andra alternativet är att använda "Playground"-alternativet som visas i panelen för finjustering (se skärmdumpen ovan), vilket startar följande _jämförande_ vy där grundmodellen och den finjusterade modellen visas sida vid sida för snabb utvärdering.

![Finjusteringsjobbstatus](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.sv.png)

Fyll helt enkelt i systemkontexten som användes i din träningsdata och ange din testfråga. Du kommer att märka att båda sidor uppdateras med samma kontext och fråga. Kör jämförelsen och du ser skillnaden i utdata mellan dem. _Observera hur den finjusterade modellen ger svaret i det format du angav i dina exempel, medan grundmodellen bara följer systemprompten_.

![Finjusteringsjobbstatus](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.sv.png)

Du kommer att märka att jämförelsen även visar antalet tokens för varje modell, samt tiden det tar för inferensen. **Detta specifika exempel är väldigt enkelt och är tänkt att visa processen, men speglar inte ett verkligt dataset eller scenario**. Du kan se att båda exemplen visar samma antal tokens (systemkontext och användarfråga är identiska) men att den finjusterade modellen tar längre tid för inferensen (egen modell).

I verkliga scenarier kommer du inte att använda ett så här enkelt exempel, utan finjustera mot riktig data (t.ex. produktkatalog för kundtjänst) där svarskvaliteten blir mycket tydligare. I _det_ sammanhanget krävs mer avancerad prompt engineering för att få motsvarande svarskvalitet med grundmodellen, vilket ökar tokenanvändningen och potentiellt även tiden för inferensen. _För att testa detta, ta en titt på finjusteringsexemplen i OpenAI Cookbook för att komma igång._



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiska översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess originalspråk ska betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi tar inget ansvar för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
